In [1]:
import json
import os
import numpy as np
import math
from typing import Dict, List, Tuple
import copy

from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector

import plotly.graph_objects as go
from plotly.subplots import make_subplots
test_projects = [
    "HomeA/2016",
    "HomeB/2016",
    "HomeC/2016",
    "HomeD/2016",
    "HomeF/2016",
    "HomeG/2016",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 20,
            },
            # "humidity#NUM" : {
            #     "range" : (0., 1.0),
            #     "interval" : 0.1,
            # },
            # "apparentTemperature#NUM" : {
            #     "range" : (-10., 100),
            #     "interval" : 10,
            # },
            "summary#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
        })
capacity = {
    "Range" : 2,
    "Microwave": 1,
    "LivingLights": 0,
    "HomeOffice": 1,
    "WashingMachine": 1,
}


In [2]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], _ = test_umass(test_project=p, ctx_info=ctx_info,train_ratio=0.7)

DEBUG:root:The number of device events from processed file: {'Range': 377, 'Microwave': 931, 'LivingLights': 1099, 'HomeOffice': 997}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 6119, 'temperature#NUM': 6119, 'humidity#NUM': 5418, 'summary#CAT': 1835}
DEBUG:root:The number of device events from processed file: {'LivingLights': 1384, 'Microwave': 619}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 6112, 'temperature#NUM': 6118, 'humidity#NUM': 5422, 'summary#CAT': 1290}
DEBUG:root:The number of device events from processed file: {'HomeOffice': 746, 'Microwave': 1425, 'LivingLights': 1283}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 6115, 'temperature#NUM': 6112, 'humidity#NUM': 5404, 'summary#CAT': 1322}
DEBUG:root:The number of device events from processed file: {'Microwave': 929, 'LivingLights': 831, 'WashingMachine': 215, 'Range': 871}
DEBUG:root:The num

ValueError: 'Breezy and Overcast' is not in list

In [ ]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [ ]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))


In [5]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

{'Range': 0, 'Microwave': 48, 'LivingLights': 0, 'HomeOffice': 79, 'WashingMachine': 23}


In [8]:
import plotly.express as px
probs = [(x["box"],x["prob"]) for x in final_conflicts["Microwave"]]
# print(probs)
# fig = px.histogram(probs)
# fig.show()

In [9]:
import copy
# Make ground truth:
ratio = 0.3
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(p)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": ctx_info,
    "capacity": capacity
}

conflict_finder = GtConflictFinder(gtconflict_cfg)
conflicts = conflict_finder.get_Gt_conflict(ctx_evts, device_events)
print(len(conflicts))

1663


In [10]:
import plotly.express as px

TEST_RATIO = 0.3
conflict_time = [x['cur_time'] for x in conflicts]
end_time = max(conflict_time)
start_time = min(conflict_time)
print(start_time)
print(end_time)
total_time_range = end_time - start_time
test_start = end_time - total_time_range * TEST_RATIO
print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}
for c in conflicts:
    if c["cur_time"] > test_start:
        conflict_device[c["device"]].append(c)
print({d:len(c) for d,c in conflict_device.items()})


2016-01-01 15:07:00
2016-12-29 16:48:00
2016-09-11 18:41:42
{'Range': 0, 'Microwave': 197, 'LivingLights': 0, 'HomeOffice': 319, 'WashingMachine': 13}


In [11]:

missed_gt_conflicts = {d:[] for d in conflict_device}
hit_gt_conflicts = {d:{} for d in conflict_device}
for d in final_conflicts:
    hit_gt_conflicts[d] = {i:0 for i in range(len(final_conflicts[d]))}
for d, conflicts in conflict_device.items():
    for c in conflicts:
        coor = ctx_info.get_coor_by_ctx(c["ctx"])
        flag = False
        for idx,c_predict in enumerate(final_conflicts[d]):
            if does_contain_point(c_predict["box"], coor):
                flag = True
                hit_gt_conflicts[d][idx] += 1
                break
        if not flag:
            missed_gt_conflicts[d].append(c)
print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
hit_count = {d:[0,0] for d in hit_gt_conflicts}
for d,hit_c in hit_gt_conflicts.items():
    for i,h in hit_c.items():
        if h == 0:
            hit_count[d][0] += 1
        else:
            hit_count[d][1] += 1
print(hit_count)

{'Range': 0, 'Microwave': 8, 'LivingLights': 0, 'HomeOffice': 31, 'WashingMachine': 0}
{'Range': [0, 0], 'Microwave': [15, 33], 'LivingLights': [0, 0], 'HomeOffice': [29, 50], 'WashingMachine': [20, 3]}


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

draw_device = "HomeOffice"
c_point = [[],[]]
missed_point = [[], []]
for c in conflict_device[draw_device]:
    coor = ctx_info.get_coor_by_ctx(c["ctx"])
    c_point[0].append(coor[0])
    c_point[1].append(coor[1])
for c in missed_gt_conflicts[draw_device]:
    coor = ctx_info.get_coor_by_ctx(c["ctx"])
    missed_point[0].append(coor[0])
    missed_point[1].append(coor[1])
fig = go.Figure(data=go.Scatter(x=c_point[0], y=c_point[1], mode="markers"))
fig.add_trace(go.Scatter(x=missed_point[0], y=missed_point[1], mode="markers", marker=dict(color="red")))

for c in final_conflicts[draw_device]:
    fig.add_shape(type="rect",
        x0=c["box"][0], y0=c["box"][1], x1=c["box"][2], y1=c["box"][3],
        line=dict(color="RoyalBlue"),
    )
fig.show()


In [11]:
hit_miss_rate = [[],[]]
for prob_threshold in np.arange(0.0, 0.5, 0.001):
    test_conflict = {
        d: [x for x in final_conflicts[d] if x["prob"] >= prob_threshold]
        for d in final_conflicts
    }
    missed_gt_conflicts = {d:[] for d in conflict_device}
    hit_gt_conflicts = {d:{} for d in conflict_device}
    for d in test_conflict:
        hit_gt_conflicts[d] = {i:0 for i in range(len(test_conflict[d]))}
    for d, conflicts in conflict_device.items():
        for c in conflicts:
            coor = ctx_info.get_coor_by_ctx(c["ctx"])
            flag = False
            for idx,c_predict in enumerate(test_conflict[d]):
                if does_contain_point(c_predict["box"], coor):
                    flag = True
                    hit_gt_conflicts[d][idx] += 1
                    break
            if not flag:
                missed_gt_conflicts[d].append(c)
    all_missed_gt_conflicts = sum([len(x) for d,x in missed_gt_conflicts.items()])
    all_gt_conflicts = sum([len(x) for d,x in conflict_device.items()])
    hit_miss_rate[0].append(1.0 - float(all_missed_gt_conflicts) / all_gt_conflicts)
    # print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
    all_pred_conflicts = 0
    all_missed_pred_conflicts = 0
    hit_count = {d:[0,0] for d in hit_gt_conflicts}
    for d,hit_c in hit_gt_conflicts.items():
        for i,h in hit_c.items():
            all_pred_conflicts += 1
            if h == 0:
                all_missed_pred_conflicts += 1
                hit_count[d][0] += 1
            else:
                hit_count[d][1] += 1
    hit_miss_rate[1].append(1.0 - float(all_missed_pred_conflicts) / all_pred_conflicts)
    # print(hit_count)

print(hit_miss_rate)

[[0.8128544423440454, 0.7863894139886578, 0.7561436672967864, 0.7353497164461248, 0.724007561436673, 0.6994328922495274, 0.6956521739130435, 0.6748582230623819, 0.6635160680529301, 0.6446124763705103, 0.6275992438563327, 0.6238185255198487, 0.6238185255198487, 0.6105860113421551, 0.5879017013232515, 0.5784499054820416, 0.5784499054820416, 0.5406427221172023, 0.5255198487712666, 0.5255198487712666, 0.5255198487712666, 0.5141776937618148, 0.5141776937618148, 0.5047258979206048, 0.4820415879017014, 0.4820415879017014, 0.4593572778827977, 0.4593572778827977, 0.4593572778827977, 0.4555765595463138, 0.4555765595463138, 0.4555765595463138, 0.4555765595463138, 0.45368620037807184, 0.45179584120982985, 0.4385633270321361, 0.4385633270321361, 0.4385633270321361, 0.43289224952741023, 0.4253308128544423, 0.4253308128544423, 0.4234404536862004, 0.41965973534971646, 0.41965973534971646, 0.41965973534971646, 0.41776937618147447, 0.4158790170132325, 0.4158790170132325, 0.4158790170132325, 0.3837429111

In [12]:
print(len(hit_miss_rate[0]))
fig = go.Figure(data=go.Scatter(x=hit_miss_rate[0], y=hit_miss_rate[1], mode="lines+markers"))
fig.show()


500


In [11]:
from rtree import index


draw = {
    "HomeA/2016": ["Range"],
    "HomeG/2016": ["Range"],
}
fig = make_subplots(rows=sum([len(draw[x]) for x in draw]), cols=1)
p = index.Property()
p.dimension = len(ctx_info.get_all_ctx_ordered())
row_id = 1
for home in draw:
    for d in draw[home]:
        r_tree = index.Index(properties=p)
        for i,x in enumerate(habit_groups[home][d]):
            bound = x["box"][0] + x["box"][1] 
            r_tree.insert(id=i, coordinates=bound, obj=sum(x["dis"]))
        on_rate_group = np.full(ctx_info.get_ctx_space_shape(), -0.1)

        for i in range(ctx_info.get_ctx_space_shape()[0]):
            for j in range(ctx_info.get_ctx_space_shape()[1]):
                result = list(r_tree.intersection([i,j,i,j], objects=True))

                on_rate_group[i,j] = result[0].object
        fig.append_trace(
            go.Heatmap(z=np.transpose(on_rate_group)),
            row_id, 1
        )
        row_id += 1
fig.show()

AttributeError: 'str' object has no attribute 'type'